# Refinitiv Data Library for Python

## Content layer - Pricing stream - Opened with a chain
This Notebook demonstrates how to retrieve level 1 streaming data (such as trades and quotes) for the constituents of a chain.  

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [login](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) to the Refinitiv Developer Community portal you will get free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding the API usage, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help.

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the Refinitiv Data Library in the _refinitiv-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _RD_LIB_CONFIG_PATH_ environment variable. The _RD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

## Some Imports to start with

In [2]:
import refinitiv.data as rd
from refinitiv.data.content import pricing
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [3]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7fcd7630abd0 {name='workspace'}>

## Retrieve data

### Get Chain consistent

In [4]:
dow = pricing.chain.Definition(name="0#.DJI").get_stream()
dow.open(with_updates=False)
dow.close()
print(dow.constituents)

['.DJI', 'AAPL.OQ', 'AMGN.OQ', 'AXP.N', 'BA.N', 'CAT.N', 'CRM.N', 'CSCO.OQ', 'CVX.N', 'DIS.N', 'DOW.N', 'GS.N', 'HD.N', 'HON.OQ', 'IBM.N', 'INTC.OQ', 'JNJ.N', 'JPM.N', 'KO.N', 'MCD.N', 'MMM.N', 'MRK.N', 'MSFT.OQ', 'NKE.N', 'PG.N', 'TRV.N', 'UNH.N', 'V.N', 'VZ.N', 'WBA.OQ', 'WMT.N']


### Create a Pricing stream with chain constituents

In [5]:
pricing_stream = pricing.Definition(universe=dow.constituents, fields=["BID", "ASK"]).get_stream()

### Define and register callbacks to receive streaming data

In [6]:
df = DataFrame()

def create_dataframe(streaming_prices):
    global df
    snapshot = streaming_prices.get_snapshot()
    field_names = snapshot.columns[1:]
    instrument_names = snapshot['Instrument'].values
    df = DataFrame(index=instrument_names, columns=field_names)
    for price in streaming_prices:
        for field_name, field_value in price.get_fields().items():
            df.at[price.name, field_name] = field_value
    display(df)


def update_dataframe(streaming_prices, instrument_name, fields):
    global df
    clear_output(wait=True)
    for field_name, field_value in fields.items():
        df.at[instrument_name, field_name] = field_value
    display(df)
    
pricing_stream.on_update(update_dataframe)
pricing_stream.on_complete(create_dataframe)

<refinitiv.data.content.pricing.Stream object at 0x7fcd763833d0 {name='['.DJI', 'AAPL.OQ', 'AMGN.OQ', 'AXP.N', 'BA.N', 'CAT.N', 'CRM.N', 'CSCO.OQ', 'CVX.N', 'DIS.N', 'DOW.N', 'GS.N', 'HD.N', 'HON.OQ', 'IBM.N', 'INTC.OQ', 'JNJ.N', 'JPM.N', 'KO.N', 'MCD.N', 'MMM.N', 'MRK.N', 'MSFT.OQ', 'NKE.N', 'PG.N', 'TRV.N', 'UNH.N', 'V.N', 'VZ.N', 'WBA.OQ', 'WMT.N']'}>

### Open the stream

In [7]:
pricing_stream.open()

,ASK,BID
.DJI,36037.17,35871.41
AAPL.OQ,179.86,179.76
AMGN.OQ,219.6,219
AXP.N,None,None
BA.N,None,None
CAT.N,None,None
CRM.N,None,None
CSCO.OQ,60.08,59.94
CVX.N,None,None
DIS.N,None,None


### Close the stream

In [8]:
pricing_stream.close()

<OpenState.Closed: 3>

## Close the session

In [9]:
rd.close_session()